# Test algorithms

## Policy Gradient

In [8]:
%run ./repos/mbzuai-rl-research/cont-pg-simple.py -b 128

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/nicolas.avila/.netrc
/home/nicolas.avila/.conda/envs/rl_energy/lib/python3.8/site-packages/wandb/sdk/lib/ipython.py:47: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML  # type: ignore


actor_loss,▂▂▂▂▂▁▂▁▁▂▁▁▁▂▂▃▄▅▇▇▇███████████████████
rollout_avg_reward,▁▁▁▁▁▁▁▁▂▂▂▂▃▃▄▄▅▇▇█████████████████████
actor_loss,3.93019
rollout_avg_reward,-1.82493
